In [1]:
import os
import ifcopenshell
import rdflib
from rdflib import Graph, Literal, RDF, URIRef, Namespace
from rdflib.graph import ConjunctiveGraph
from rdflib.plugins.stores.memory import Memory
from rdflib.namespace import XSD
from pprint import pprint
from utils import ifc2geom
import trimesh
import re
import uuid

## Completing raw LBD Graphs for object geometries

- Create mesh geometries for ifc objects
- Export as PLY files
- supplement PLY file paths to object nodes in LBD Graph
- Extract bounding box from mesh, supplement to object nodes in LBD Graph

In [2]:
BOT = Namespace('https://w3id.org/bot#')

# Specify the model to enrich
for model_name in ['ms_arc', 'ms_str', 'ms_plum']:
    model = ifcopenshell.open('ifc_models/' + model_name + '.ifc')
    model_ns = Namespace('https://example.org/' + model_name + '#')

    # parse the raw LBD graph generated from the IFC models
    g = Graph()
    g.parse(model_name + '_LBD.ttl')
    
    # query for all building objects from the LBD graph
    query_for_guid ="""
        SELECT ?uri ?guid
        WHERE {
            ?uri rdf:type bot:Element .
            ?uri props:globalIdIfcRoot_attribute_simple ?guid .
        }"""

    qres = g.query(query_for_guid)
    
    # loop through building objects and enrich with geometry info
    for row in qres:
        uri = row.uri
        guid_short = row.guid
        guid_long = uri.split('_')[-1]
        element = model[guid_short]

        # 1.extract raw mesh data for element (ifcopenshell.geom)
        # 2.create mesh object (trimesh)
        # 3.export as <element_guid>.ply file
        # 4.return path to .ply file
        geom_path = ifc2geom.export_exact_geometry(guid_long, model, element, model_name, check_aggregate=False)

        # 1.load <element_guid>.ply as mesh object (trimesh)
        # 2.generate bbox for the mesh object (trimesh)
        # 3.return bbox parameters as string in the form [x, y, z, dim_x, dim_y, dim_z]
        bbox_str = ifc2geom.bbox_to_string(guid_long, model_name)

        # if success, add geomertry information to graph
        if not bbox_str is None and not geom_path is None:
            g.add((URIRef(uri), BOT.hasSimple3DModel, Literal(bbox_str)))
            g.add((URIRef(uri), BOT.has3DModel, Literal(geom_path)))
    g.bind(model_name.split('_')[-1], model_ns)
    g.serialize(destination='enriched_' + model_name + '_LBD.ttl')

Geometry exist for 4c35a2e4-de2e-4015-bf04-beea04d6892e
Geometry exist for 69d19b99-7258-4d69-8af0-c79422b77872
Geometry exist for 69d19b99-7258-4d69-8af0-c79422b55b21
Geometry exist for 69d19b99-7258-4d69-8af0-c79422b7786c
Geometry exist for 69d19b99-7258-4d69-8af0-c79422b77843
Geometry exist for 69d19b99-7258-4d69-8af0-c79422b77868
Geometry exist for 69d19b99-7258-4d69-8af0-c79422b77871
Geometry exist for 69d19b99-7258-4d69-8af0-c79422b77877
Geometry exist for 63fce307-79e4-4db6-a098-ecb90d2207f0
Geometry exist for 69d19b99-7258-4d69-8af0-c79422b55ba1
Geometry exist for 69d19b99-7258-4d69-8af0-c79422b5580b
Geometry exist for 69d19b99-7258-4d69-8af0-c79422b5404b
Geometry exist for 69d19b99-7258-4d69-8af0-c79422b543fe
Geometry exist for 69d19b99-7258-4d69-8af0-c79422b555d3
No geometry found for 69d19b99-7258-4d69-8af0-c79422b54c15, skipping
Geometry file not found when creating bounding box for 69d19b99-7258-4d69-8af0-c79422b54c15
Geometry exist for 69d19b99-7258-4d69-8af0-c79422b54106

## Inter-domain relationships

### Combining the completed subgraphs into one (meta-) conjunction graph

In [3]:
ex_ns = Namespace('https://example.org/domains#')
store = Memory()

conj_graph = ConjunctiveGraph(store=store)
conj_graph.bind('domain', ex_ns)

arc_graph = Graph(store=store, identifier=ex_ns.architecture)
str_graph = Graph(store=store, identifier=ex_ns.structure)
plum_graph = Graph(store=store, identifier=ex_ns.plumbing)
cbim_graph = Graph(store=store, identifier=ex_ns.CBIM)

arc_graph.parse('enriched_ms_arc_LBD.ttl')
str_graph.parse('enriched_ms_str_LBD.ttl')
plum_graph.parse('enriched_ms_plum_LBD.ttl')

<Graph identifier=https://example.org/domains#plumbing (<class 'rdflib.graph.Graph'>)>

### Exhaustive search over objects to establish CBIM:RelSpatial & CBIM:equivalentTo

In [4]:
query_for_prop = """
    SELECT ?uri ?guid ?type ?bbox ?path (GROUP_CONCAT(DISTINCT ?class; SEPARATOR="#") AS ?classes) 
    WHERE {
        FILTER(STRSTARTS(STR(?uri), "{uri}"))
        ?uri props:globalIdIfcRoot_attribute_simple ?guid .
        OPTIONAL {?uri props:objectTypeIfcObject_attribute_simple ?type .}
        ?uri rdf:type ?class .
        ?uri bot:hasSimple3DModel ?bbox .
        ?uri bot:has3DModel ?path .
        
    }
    GROUP BY ?uri
    """
qres_arc = conj_graph.query(query_for_prop.replace("{uri}", 'https://example.org/ms_arc#'))
qres_str = conj_graph.query(query_for_prop.replace("{uri}", 'https://example.org/ms_str#'))
qres_plum = conj_graph.query(query_for_prop.replace("{uri}", 'https://example.org/ms_plum#'))

#### Defining parameters

In [5]:
# set of keywords considered as being equivalent
equivalent_class = {'wall', 'panel', 'partition'}
add_topo = True
proximity = 500

#### Compare each object pair, instantiate RelSpatial and equivalence relationships

In [6]:
CBIM = Namespace('http://example.org/CBIM_Ontology#')
conj_graph.bind('CBIM', CBIM)

for qres1, qres2 in [(qres_arc, qres_str), (qres_arc, qres_plum)]:
    print('\n==================================')
    # looping through returned list of architectural elements
    for elem1 in qres1:
        # looping through returned list of strcutural elements
        for elem2 in qres2:

            # compute topological relationship based on bbox
            topo_res, topo_offset = ifc2geom.topology_rel(elem1.bbox, elem2.bbox)

            # establish CBIM topology relationships only if bbox in proximity
            if max([abs(x) for x in topo_offset.values()]) < proximity and add_topo:
                print(f"Spatial relationship established between {elem1.uri}, {elem2.uri}")

                # initializing new CBIM:RelSpatial instance
                uid = uuid.uuid4()
                rel_spatial = getattr(CBIM, 'RelSpatial_' + uid.urn[9:])
                topology = ', '.join(e for e in [topo_res[x] for x in ['x', 'y', 'z']])
                offset = ', '.join(str(e) for e in [topo_offset[x] for x in ['x', 'y', 'z']])

                # attach object properties to CBIM:RelSpatial instance
                cbim_graph.add((rel_spatial, RDF.type, CBIM.RelSpatial))
                cbim_graph.add((rel_spatial, CBIM.hasSubject, elem1.uri))
                cbim_graph.add((rel_spatial, CBIM.hasObject, elem2.uri))

                # attach datatype properties to CBIM:RelSpatial instance
                cbim_graph.add((rel_spatial, CBIM.topology, Literal(topology)))
                cbim_graph.add((rel_spatial, CBIM.offset, Literal(offset)))
                cbim_graph.add((rel_spatial, CBIM.inContact, Literal(False, datatype=XSD.boolean)))

            # if bounding boxes overlap, countinue investigating the exact geometries
            if max([abs(x) for x in topo_offset.values()]) == 0:

                # computation based on exact geometries
                is_contact, contact_res = ifc2geom.test_for_contact(elem1.path, elem2.path)
                cbim_graph.set((rel_spatial, CBIM.inContact, Literal(is_contact, datatype=XSD.boolean)))

                # extracting keywords 
                arc_set = set(elem1.classes.toPython().lower().split('#')).union(set(re.split('_|-|:| ', elem1.type.toPython().lower())))
                str_set = set(elem2.classes.toPython().lower().split('#')).union(set(re.split('_|-|:| ', elem2.type.toPython().lower())))

                # elements from equivalent classes, chances for being equivalentTo
                if len(arc_set.intersection(equivalent_class)) > 0 and len(str_set.intersection(equivalent_class)) > 0:               

                    # test for intersecting volume
                    if contact_res['intersection_volume']/contact_res['subject_volume'] > 0.8 or \
                    contact_res['intersection_volume']/contact_res['object_volume'] > 0.8:
                        print(f"\n\t equivalentTo relationship established between {elem1.uri}, {elem2.uri}\n")
                        cbim_graph.add((elem1.uri, CBIM.equivalentTo, elem2.uri))
                        cbim_graph.add((elem2.uri, CBIM.equivalentTo, elem1.uri))



Spatial relationship established between https://example.org/ms_arc#railing_4c35a2e4-de2e-4015-bf04-beea04d6892e, https://example.org/ms_str#slab_a2450c0e-392d-4c89-b2b5-3825392cf526
Spatial relationship established between https://example.org/ms_arc#railing_4c35a2e4-de2e-4015-bf04-beea04d6892e, https://example.org/ms_str#slab_a7d9c1db-a96d-4ded-9a32-f69dddef99b9
Spatial relationship established between https://example.org/ms_arc#railing_4c35a2e4-de2e-4015-bf04-beea04d6892e, https://example.org/ms_str#wall_2fcd0ec3-3467-4059-af15-126801a3c442
Spatial relationship established between https://example.org/ms_arc#railing_4c35a2e4-de2e-4015-bf04-beea04d6892e, https://example.org/ms_str#wall_97bfe34b-6deb-45ec-9ddd-fffac56fb1d8
Spatial relationship established between https://example.org/ms_arc#slab_69d19b99-7258-4d69-8af0-c79422b54192, https://example.org/ms_str#buildingelement_3338f5cb-0e8f-4fd0-bd42-99428c302460
Spatial relationship established between https://example.org/ms_arc#slab_69d

#### Serialize enriched conjunction graph to ms_conjuction_enriched.trig

In [8]:
conj_graph.serialize(destination='enriched_ms_metagraph.trig', format='trig')

<Graph identifier=N263c2daa33574db2b90d813c627534d6 (<class 'rdflib.graph.ConjunctiveGraph'>)>